In [1]:
import tools
import importlib
import memory
importlib.reload(tools)
importlib.reload(memory)

from tools import network_tools
from tools import firewall_tools

In [8]:
alerts = await network_tools.get_fast_alerts(time_window=2)
alerts

{'security_events': {'alerts': [],
  'count': 0,
  'time_window_minutes': 2,
  'timestamp': '2025-08-06T21:26:45.228068'}}

In [3]:
network_tools.get_docker_containers()

{'honeypot_config': [{'id': '640353ced621',
   'name': 'cve-2014-6271-web-1',
   'image': 'vulhub/bash:4.3.0-with-httpd',
   'status': 'running',
   'created': '2025-08-06T07:48:02.488925808Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'}]}

In [16]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    DROP       all  --  192.168.100.0/24     172.20.0.0/24       
5    DROP       all  --  172.20.0.0/24        192.168.100.0/24    
6    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [15]:
await firewall_tools.remove_firewall_rule([1])

INFO:tools.firewall_tools:Removing firewall rules: [1]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.2']}

In [13]:
honeypot_ip = '172.20.0.2' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.2 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.2 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.2 -> 192.168.100.2/24 (tcp)'}